# Install requirements for selenium

In [13]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


# Import modules

In [14]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
import json
import re

# Request page source from URL

In [15]:
# Identitas browser saat akses halaman web karena IMDb menolak (403 Forbidden) jika tidak ada user-agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"

In [16]:
# Keperluan untuk penggunaan selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--diable-dve-shm-uage')
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=options)

In [17]:
driver.get('https://www.imdb.com/list/ls507147151/')

In [18]:
time.sleep(5)  # tunggu JavaScript selesai load

# Ambil page source dan parse dengan BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")

# Print seluruh HTML dengan struktur yang rapi
print(soup.prettify())

<html class="scriptsOn" lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/I/216YVwoRFDL.js">
  </script>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <script>
   if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }
  </script>
  <title>
   IMDB's Top 100 Video Games of All Time
  </title>
  <meta content="minimum 1k ratings, must be listed as a video game on imdb" data-id="main" name="description"/>
  <meta content="0cadf7898134e79b" name="google-site-verification"/>
  <meta content="C1DACEF2769068C0B0D2687C9E5105FA" name="msvalidate.01"/>
  <meta content="max-image-preview:large" name="robots"/>
  <script type="application/ld+json">
   {"@type":"ItemList","itemListElement":[{"@type":"ListItem","item":{"@type":"VideoGame","url":"https://www.imdb.com/title/tt6161168/","name":"Red

# Scraped the data

In [19]:
# Dapatkan konten html yang secara khusus memakai tag li dan class yang bersesuaian (lokasi dimana informasi data yang ingin dicari, dilihat dari inspect)
scraped_videogames = soup.find_all('li', class_='ipc-metadata-list-summary-item')
scraped_videogames

[<li class="ipc-metadata-list-summary-item"><div class="ipc-metadata-list-summary-item__c"><div class="ipc-metadata-list-summary-item__tc"><span aria-disabled="false" class="ipc-metadata-list-summary-item__t ipc-btn--not-interactable"></span><div class="sc-52ea7f05-1 ejhgHs dli-parent"><div class="sc-52ea7f05-0 YKXtw"><div class="sc-d0224b4e-0 jfogmY dli-poster-container"><div class="ipc-poster ipc-poster--base ipc-poster--media-radius ipc-poster--wl-true ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group"><div class="ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--media-radius ipc-media--base ipc-media--poster-s ipc-poster__poster-image ipc-media__img" style="width:100%"><img alt="Roger Clark in Red Dead Redemption II (2018)" class="ipc-image" loading="lazy" sizes="50vw, (min-width: 480px) 34vw, (min-width: 600px) 26vw, (min-width: 1024px) 16vw, (min-width: 1280px) 16vw" src="https://m.media-amazon.com/images/M/MV5BNzRhZ

In [20]:
# Cek data yang didapat
all_target_li = soup.find_all('li', class_='ipc-metadata-list-summary-item')
print(f"Total yang cocok: {len(all_target_li)}")

Total yang cocok: 100


In [21]:
# Untuk clean penomoran di title
def clean_title(title):
    return re.sub(r"^\d+\.\s*", "", title).strip()

In [ ]:
video_game_list = []
director_list = []
star_list = []
star_id_counter = 1
director_id_counter = 1

# Untuk pencocokan nama ke ID 
director_dict = {}
star_dict = {}

for li in scraped_videogames:
    try:
        # Judul
        title_tag = li.find("h3", class_="ipc-title__text")
        raw_title = title_tag.get_text(strip=True) if title_tag else None
        title = clean_title(raw_title)

        # Tahun, Age Rating 
        metadata_spans = li.select(".dli-title-metadata span")
        year = metadata_spans[0].get_text(strip=True) if len(metadata_spans) > 0 else None
        age_rating = metadata_spans[1].get_text(strip=True) if len(metadata_spans) > 1 else None

        # Rating (IMDb)
        rating_tag = li.select_one("span[class*='ipc-rating-star--rating']")
        rating = rating_tag.get_text(strip=True) if rating_tag else None

        # Deskripsi
        desc_tag = li.select_one(".title-description-plot-container .ipc-html-content-inner-div")
        description = desc_tag.get_text(strip=True) if desc_tag else None

        # Director(s)
        director_ids = []
        director_tags = li.select("span.sc-9d52d06f-3")
        for tag in director_tags:
            if tag.get_text(strip=True) in ["Director", "Directors"]:
                for sibling in tag.find_next_siblings("span", class_="title-description-credit"):
                    a_tag = sibling.find("a")
                    if a_tag:
                        name = a_tag.get_text(strip=True)
                        if name not in director_dict:
                            director_dict[name] = director_id_counter
                            director_list.append({
                                "id": director_id_counter,
                                "name": name
                            })
                            director_id_counter += 1
                        director_ids.append(director_dict[name])
                break

        # Stars
        star_ids = []
        for tag in director_tags:
            if tag.get_text(strip=True) == "Stars":
                for sibling in tag.find_next_siblings("span", class_="title-description-credit"):
                    a_tag = sibling.find("a")
                    if a_tag:
                        name = a_tag.get_text(strip=True)
                        if name not in star_dict:
                            star_dict[name] = star_id_counter
                            star_list.append({
                                "id": star_id_counter,
                                "name": name
                            })
                            star_id_counter += 1
                        star_ids.append(star_dict[name])
                break

        # Simpan game
        video_game_list.append({
            "title": title,
            "year": year,
            "age rating": age_rating,
            "rating": rating,
            "description": description,
            "directors": director_ids,
            "stars": star_ids
        })

    except Exception as e:
        print(f"Error processing video game: {e}")

In [ ]:
# Lihat seluruh list director yang didapatkan
for director in director_list:
    print(director)

{'id': 1, 'name': 'Neil Druckmann'}
{'id': 2, 'name': 'Bruce Straley'}
{'id': 3, 'name': 'Konrad Tomaszkiewicz'}
{'id': 4, 'name': 'Cory Barlog'}
{'id': 5, 'name': 'Hideo Kojima'}
{'id': 6, 'name': 'Eiji Aonuma'}
{'id': 7, 'name': 'Yoichi Yamada'}
{'id': 8, 'name': 'Yoshiaki Koizumi'}
{'id': 9, 'name': 'Casey Hudson'}
{'id': 10, 'name': 'Yoshinori Kitase'}
{'id': 11, 'name': 'Yasushi Matsumura'}
{'id': 12, 'name': 'Masato Yagi'}
{'id': 13, 'name': 'Akihiko Matsui'}
{'id': 14, 'name': 'Takashi Tokita'}
{'id': 15, 'name': 'Todd Howard'}
{'id': 16, 'name': 'Sefton Hill'}
{'id': 17, 'name': 'Hidemaro Fujibayashi'}
{'id': 18, 'name': 'Hiroki Hirano'}
{'id': 19, 'name': 'Kentaro Tominaga'}
{'id': 20, 'name': 'Amy Hennig'}
{'id': 21, 'name': 'David Speyrer'}
{'id': 22, 'name': 'Takayuki Kobayashi'}
{'id': 23, 'name': 'Masashi Tsuboyama'}
{'id': 24, 'name': 'Feargus Urquhart'}
{'id': 25, 'name': 'Nate Fox'}
{'id': 26, 'name': 'Katsura Hashino'}
{'id': 27, 'name': 'Guillermo del Toro'}
{'id': 2

In [ ]:
# Lihat seluruh list star yang didapatkan
for star in star_list:
    print(star)

{'id': 1, 'name': 'Benjamin Byron Davis'}
{'id': 2, 'name': 'Roger Clark'}
{'id': 3, 'name': 'Rob Wiethoff'}
{'id': 4, 'name': 'Troy Baker'}
{'id': 5, 'name': 'Ashley Johnson'}
{'id': 6, 'name': 'Hana Hayes'}
{'id': 7, 'name': 'Doug Cockle'}
{'id': 8, 'name': 'Denise Gough'}
{'id': 9, 'name': 'Jo Wyatt'}
{'id': 10, 'name': 'Christopher Judge'}
{'id': 11, 'name': 'Sunny Suljic'}
{'id': 12, 'name': 'Jeremy Davies'}
{'id': 13, 'name': 'David Hayter'}
{'id': 14, 'name': 'Patric Zimmerman'}
{'id': 15, 'name': 'Peter Lurie'}
{'id': 16, 'name': 'Nobuyuki Hiyama'}
{'id': 17, 'name': 'Fujiko Takimoto'}
{'id': 18, 'name': 'Jun Mizusawa'}
{'id': 19, 'name': 'Shawn Fonteno'}
{'id': 20, 'name': 'Ned Luke'}
{'id': 21, 'name': 'Steven Ogg'}
{'id': 22, 'name': 'Shohreh Aghdashloo'}
{'id': 23, 'name': 'Maggie Baird'}
{'id': 24, 'name': 'Adam Baldwin'}
{'id': 25, 'name': 'Nolan North'}
{'id': 26, 'name': 'Richard McGonagle'}
{'id': 27, 'name': 'Jodi Benson'}
{'id': 28, 'name': 'Jim Piddock'}
{'id': 29, 

# Store the Scraped Data

In [25]:
# Simpan ke file json
with open("../data/videogames.json", "w", encoding="utf-8") as f:
    json.dump(video_game_list, f, ensure_ascii=False, indent=4)

with open("../data/directors_videogames.json", "w", encoding="utf-8") as f:
    json.dump(director_list, f, ensure_ascii=False, indent=4)

with open("../data/stars_videogames.json", "w", encoding="utf-8") as f:
    json.dump(star_list, f, ensure_ascii=False, indent=4)